Fraud Detection Mechanism
1. 3TSS (3TimeSameStores) - Every Mapclub member makes 3 Transactions at the same store in 1 day.
2. E150K (Earn More 150k) - Every Mapclub member who makes 1 Transaction and gets 150K points or more.
3. 1T3DSS (1TimeIn3DaySameStores) - Every Mapclub member who makes 1 transaction at the same store within 3 consecutive days.
4. 4TDS (4TimeDifferentStoreSameDay) - Every Mapclub member who makes 4 transactions at different stores in 1 day.
5. 1ADC (1AreaDifferentConcept) - Every Mapclub member who makes transactions in 1 area and has 3 more diffrent concepts in 1 day.
6. SCDS (SameConceptDiffrentStore) - Every Mapclub member who makes transactions in the same concept but with a diffrent store code in 1 day.

In [ ]:

import numpy as np
import pandas as pd
import re, gc
import time
import datetime

import sqlalchemy as sql
from sqlalchemy.sql import text
from sqlalchemy.pool import NullPool
from urllib.parse import quote
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

from pandasql import sqldf
from dateutil import rrule


In [1]:
username = ""
password = ""
host = "" # new

connect_string_dwhdb = 'mysql://'+username+':'+quote(password)+'@'+host+'/?charset=utf8mb4'
sql_engine_dwhdb = sql.create_engine(connect_string_dwhdb, poolclass=NullPool)
sql_engine_dwhdb.connect()

## Chosee what date to process manually (date_snap)

In [3]:
# tahon, bulan, tanggal
date_obj = datetime.date(2023, 11, 17)
date_obj

In [4]:
# date_str = "11 September 2023"
# date_obj = datetime.datetime.strptime(date_str, "%d %B %Y").date()
# date_obj

In [5]:
partition_to_execute = 'SP{0}'.format(date_obj.strftime("%Y%m")) + ',' + 'SP{0}'.format((date_obj + datetime.timedelta(days=-3)).strftime("%Y%m"))

partition_to_execute

In [6]:

end_date_trx = date_obj.strftime("%Y-%m-%d")
start_date_trx = (date_obj + datetime.timedelta(days=-3)).strftime("%Y-%m-%d")

In [7]:
start_date_trx

In [8]:
end_date_trx

## Raw Data

In [10]:
q = f""" 
SELECT a.internalreference
FROM mardiyan.LoyaltyMemberWalletTransactions PARTITION({partition_to_execute}) as a
WHERE a.type LIKE "%cancel%" AND CAST(a.TransactionTime AS DATE) BETWEEN '{start_date_trx}' AND '{end_date_trx}' 
"""
gclub_Cancel = pd.read_sql(text(q), sql_engine_dwhdb)

q = (f""" 
SELECT a.MemberID, a.LoyaltyMemberWalletTransactionID, a.`Type`, a.TransactionTime, a.LocationReference, b.ConceptName, b.SBUGroup,
b.Area, b.Province, b.City, a.PointValue, a.Reference, c.ChannelReference,
c.LoyaltyPayServicePayTransactionID, c.RuleServiceTransactionID, c.WalletServiceEarnTransactionID, c.WalletServiceBurnTransactionID,
d.RetailValueBeforeTax, ROUND(SUM(e.RetailValueBeforeTax),0) AS RetailValueBeforeTaxItem,
if(d.RetailValueBeforeTax < 1000,IFNULL(ROUND(SUM(e.RetailValueBeforeTax),0),d.RetailValueBeforeTax), d.RetailValueBeforeTax) AS NewRetailValueBeforeTax, c.RetailValueAfterTax,
case when right(a.LocationReference,2) = 'df' then 'MonoBrand'
when a.LocationReference IN ('M901') then 'MonoBrand'
when a.LocationReference IN ('M902') then 'DIGIMAP Shopee'
when c.ChannelReference IN ('MAPEMALL') then 'Mapemall'
ELSE 'store' END AS typeStore
FROM mardiyan.LoyaltyMemberWalletTransactions PARTITION({partition_to_execute}) AS a
LEFT JOIN mardiyan.MasterStoresV2 AS b ON a.LocationReference = b.StoreCode 
-- if(a.LocationReference = 'CV85', 'CV40', a.LocationReference) = b.StoreCode 
AND CAST(a.TransactionTime AS DATE)
    BETWEEN CAST(b.ValidFrom AS DATE) AND CAST(b.ValidUntil AS DATE)
LEFT JOIN mardiyan.LoyaltyPayServicePayTransactions PARTITION({partition_to_execute}) AS c ON a.LoyaltyMemberWalletTransactionID = c.WalletServiceEarnTransactionID 
LEFT JOIN mardiyan.LoyaltyRuleServiceTransactionResults PARTITION({partition_to_execute}) AS d ON c.RuleServiceTransactionID = d.TransactionID
LEFT JOIN mardiyan.LoyaltyRuleServiceTransactionBasketItems PARTITION({partition_to_execute}) AS e ON c.RuleServiceTransactionID = e.TransactionID
WHERE a.`Type` = 'EARN' AND a.InternalReference IS NULL AND CAST(a.TransactionTime AS DATE) -- BETWEEN '2023-04-24' AND '2023-04-30'
    BETWEEN '{start_date_trx}' AND '{end_date_trx}'  
    -- BETWEEN (SELECT DATE_SUB(CURDATE(),INTERVAL 3 DAY)) AND (SELECT DATE_SUB(CURDATE(),INTERVAL 1 DAY)) 
-- AND a.LoyaltyMemberWalletTransactionID NOT IN (SELECT internalreference FROM gclub.cancel) -- AND date(a.CreatedAt) <= '2021-12-05'
GROUP BY a.LoyaltyMemberWalletTransactionID, c.LoyaltyPayServicePayTransactionID, c.RuleServiceTransactionID
ORDER BY a.TransactionTime
;
""")
gclub_Earn = pd.read_sql(text(q), sql_engine_dwhdb, parse_dates=['TransactionTime'])

q = (f"""
SELECT a.MemberID, a.LoyaltyMemberWalletTransactionID, a.`Type`, a.TransactionTime, a.LocationReference, b.ConceptName, b.SBUGroup,
b.Area, b.Province, b.City, a.PointValue, a.Reference, c.ChannelReference,
c.LoyaltyPayServicePayTransactionID, c.RuleServiceTransactionID, c.WalletServiceEarnTransactionID, c.WalletServiceBurnTransactionID,
d.RetailValueBeforeTax, ROUND(SUM(e.RetailValueBeforeTax),0) AS RetailValueBeforeTaxItem,
if(d.RetailValueBeforeTax < 1000,IFNULL(ROUND(SUM(e.RetailValueBeforeTax),0),d.RetailValueBeforeTax), d.RetailValueBeforeTax) AS NewRetailValueBeforeTax, c.RetailValueAfterTax,
case when right(a.LocationReference,2) = 'df' then 'MonoBrand'
when a.LocationReference IN ('M901') then 'MonoBrand'
when a.LocationReference IN ('M902') then 'DIGIMAP Shopee'
when c.ChannelReference IN ('MAPEMALL') then 'Mapemall'
ELSE 'store' END AS typeStore
FROM mardiyan.LoyaltyMemberWalletTransactions PARTITION({partition_to_execute}) AS a
LEFT JOIN mardiyan.MasterStoresV2 AS b ON a.LocationReference = b.StoreCode 
-- if(a.LocationReference = 'CV85', 'CV40', a.LocationReference) = b.StoreCode 
AND CAST(a.TransactionTime AS DATE)
    BETWEEN CAST(b.ValidFrom AS DATE) AND CAST(b.ValidUntil AS DATE)
LEFT JOIN mardiyan.LoyaltyPayServicePayTransactions PARTITION({partition_to_execute}) AS c ON a.LoyaltyMemberWalletTransactionID = c.WalletServiceBurnTransactionID 
LEFT JOIN mardiyan.LoyaltyRuleServiceTransactionResults PARTITION({partition_to_execute}) AS d ON c.RuleServiceTransactionID = d.TransactionID
LEFT JOIN mardiyan.LoyaltyRuleServiceTransactionBasketItems PARTITION({partition_to_execute}) AS e ON c.RuleServiceTransactionID = e.TransactionID
WHERE a.`Type` = 'BURN' AND a.InternalReference IS NULL AND CAST(a.TransactionTime AS DATE) -- BETWEEN '2023-04-24' AND '2023-04-30'
    BETWEEN '{start_date_trx}' AND '{end_date_trx}'  
    -- BETWEEN (SELECT DATE_SUB(CURDATE(),INTERVAL 3 DAY)) AND (SELECT DATE_SUB(CURDATE(),INTERVAL 1 DAY)) 
-- AND a.LoyaltyMemberWalletTransactionID NOT IN (SELECT internalreference FROM gclub.cancel) -- AND date(a.CreatedAt) <= '2021-12-05'
GROUP BY a.LoyaltyMemberWalletTransactionID, c.LoyaltyPayServicePayTransactionID, c.RuleServiceTransactionID
ORDER BY a.TransactionTime
;
""")
gclub_Burn = pd.read_sql(text(q), sql_engine_dwhdb, parse_dates=['TransactionTime'])

gclub_Earn = gclub_Earn[~(gclub_Earn['LoyaltyMemberWalletTransactionID'].isin(gclub_Cancel['internalreference']))]
gclub_Burn = gclub_Burn[~(gclub_Burn['LoyaltyMemberWalletTransactionID'].isin(gclub_Cancel['internalreference']))]

q = """
SELECT * FROM -- PROSES FILTER SALES VALUE TRANSACTION DARI TRANSACTION BURN (HANYA AMBIL YG FULL REDEEM)
(	SELECT a.MemberID, a.LoyaltyMemberWalletTransactionID, a.`Type`, a.TransactionTime, a.LocationReference, a.ConceptName, a.SBUGroup,
	a.Area, a.Province, a.City, a.PointValue, a.Reference, a.ChannelReference,
	a.LoyaltyPayServicePayTransactionID, a.RuleServiceTransactionID, a.WalletServiceEarnTransactionID, a.WalletServiceBurnTransactionID,
	a.RetailValueBeforeTax, a.RetailValueBeforeTaxItem, 
	IIF(a.NewRetailValueBeforeTax <= 0, ROUND(a.RetailValueAfterTax / 1.1), 
		IIF(a.NewRetailValueBeforeTax IS NULL, ROUND(a.RetailValueAfterTax / 1.1), a.NewRetailValueBeforeTax)) AS NewRetailValueBeforeTax, a.RetailValueAfterTax, a.typeStore 
	FROM gclub_Burn AS a
	LEFT JOIN gclub_Earn AS b ON a.MemberID = b.MemberID AND a.LocationReference = b.LocationReference AND DATE(a.TransactionTime) = DATE(b.TransactionTime)
		-- AND CAST(a.TransactionTime AS DATE) = CAST(b.TransactionTime AS DATE)
	WHERE 
	IIF(-- KONDISI UNTUK MEMFILTER TRANSAKSI FULL REDEEM
		a.MemberID = b.MemberID AND a.LocationReference = b.LocationReference AND DATE(a.TransactionTime) = DATE(b.TransactionTime),
			-- AND CAST(a.TransactionTime AS DATE) = CAST(b.TransactionTime AS DATE),
			IIF(a.WalletServiceEarnTransactionID IS NOT NULL AND a.Reference IS NOT NULL, 'No', -- pli
				IIF(a.WalletServiceEarnTransactionID IS NULL AND a.Reference IS NULL, 'No', 'Yes')), 'Yes') = 'Yes'
	GROUP BY a.MemberID, a.LoyaltyMemberWalletTransactionID, a.TransactionTime
	ORDER BY a.MemberID, a.LoyaltyMemberWalletTransactionID, a.TransactionTime
) AS aa 
;
"""
gclub_TempSalesTrxValueEarnBurnA = sqldf(q, locals())

gclub_TempSalesTrxValueEarnBurn = pd.concat([gclub_Earn, gclub_TempSalesTrxValueEarnBurnA])
gclub_TempSalesTrxValueEarnBurn['TransactionTime'] = pd.to_datetime(gclub_TempSalesTrxValueEarnBurn['TransactionTime'])

q = """ 
SELECT a.MemberID, a.RuleServiceTransactionID, a.`Type`, a.LocationReference, a.SBUGroup, a.ConceptName, a.TransactionTime,
strftime('%m', a.TransactionTime) AS Month, strftime('%Y', a.TransactionTime) AS YEAR,
-- MONTH() , YEAR(a.TransactionTime) ,
case
	when LENGTH(a.Reference) = 43 then 'NO'
	when a.Reference LIKE '%adj%' then 'NO'
	when a.Reference LIKE '%inject%' then 'NO'
	when a.Reference LIKE '%point%' then 'NO'
	when a.Reference LIKE '%IGStory%' then 'NO'
	when a.Reference LIKE '%test%' then 'NO'
	when a.Reference IS NULL AND a.NewRetailValueBeforeTax = 0 then 'NO'
	-- when a.NewRetailValueBeforeTax = 0 then 'NO'
	when a.NewRetailValueBeforeTax is NULL then 'NO'	
	ELSE 'YES' 
END AS Keterangan,
a.RetailValueBeforeTax, a.RetailValueBeforeTaxItem, a.NewRetailValueBeforeTax, a.PointValue, a.Reference, a.ChannelReference, a.Area, 
a.City, a.Province, a.TypeStore
FROM gclub_TempSalesTrxValueEarnBurn AS a
WHERE -- a.SBUGroup IS NULL AND 
case
	when LENGTH(a.Reference) = 43 then 'NO'
	when a.Reference LIKE '%adj%' then 'NO'
	when a.Reference LIKE '%inject%' then 'NO'
	when a.Reference LIKE '%point%' then 'NO'
	when a.Reference LIKE '%IGStory%' then 'NO'
	when a.Reference LIKE '%test%' then 'NO'
	when a.Reference IS NULL AND a.NewRetailValueBeforeTax = 0 then 'NO'
	-- when a.NewRetailValueBeforeTax = 0 then 'NO'
	when a.NewRetailValueBeforeTax is NULL then 'NO'	
	ELSE 'YES' 
END = 'YES' 
-- GROUP BY  a.SBUGroup, a.ConceptName, MONTH(a.TransactionTime), YEAR(a.TransactionTime)
;
"""
gclub_SalesTrxValueEarnBurn = sqldf(q, locals())

gclub_SalesTrxValueEarnBurn['TransactionTime'] = pd.to_datetime(gclub_SalesTrxValueEarnBurn['TransactionTime'])

gclub_SalesTrxValueEarnBurn.to_sql(con=sql_engine_dwhdb,
    name='WeeklyChannelFraud',
    schema='gclub', # re-check
    index=False,
    if_exists='replace')

del gclub_Cancel
del gclub_Earn
del gclub_Burn
del gclub_TempSalesTrxValueEarnBurnA
del gclub_TempSalesTrxValueEarnBurn

## Process Data
### Raw Data
q = f""" 
SELECT a.MemberID, c.LoyaltyMemberID, CONCAT(c.FirstName, " ", c.LastName) MemberName, 
	DATE(c.BirthDate) BirthDate, c.Address1, c.Address2, c.Phone, c.Email,
	DATE(c.EnrollmentDate) EnrollmentDate, c.EnrollmentLocationReference EnrollmentStore,
	d.TotalPointsEarned, d.TotalPointsBurned, DATE(a.TransactionTime) TransactionTime, a.LocationReference StoreLocation, 
	b.EarnedPoints EarnedPoints, a.RuleServiceTransactionID, a.NewRetailValueBeforeTax
FROM gclub.WeeklyChannelFraud AS a
LEFT JOIN mardiyan.LoyaltyRuleServiceTransactionResults PARTITION({partition_to_execute}) AS b 
	ON a.RuleServiceTransactionID = b.TransactionID
LEFT JOIN mardiyan.LoyaltyMemberEnrollmentServiceMembers AS c ON 
	a.MemberID = c.LoyaltyMemberEnrollmentServiceMemberID
LEFT JOIN mardiyan.LoyaltyMemberWallets d
	ON a.MemberID = d.LoyaltyMemberWalletID 
;
"""
dx = pd.read_sql_query(text(q), sql_engine_dwhdb, parse_dates=['BirthDate', 'EnrollmentDate', 'TransactionTime'])

In [11]:

df = dx[(pd.to_datetime(dx['TransactionTime'].dt.date) >= dx['TransactionTime'].max()) & (pd.to_datetime(dx['TransactionTime'].dt.date) <= dx['TransactionTime'].max())].copy()
dfC = dx[(pd.to_datetime(dx['TransactionTime'].dt.date) >= dx['TransactionTime'].min()) & (pd.to_datetime(dx['TransactionTime'].dt.date) <= dx['TransactionTime'].max())].copy()
dfC.sort_values('TransactionTime', inplace=True)

## Process Data Point

### Point A

In [14]:

### Point A - Thor - 3TimeSameStores
q = """ 
SELECT a.MemberID, a.LoyaltyMemberID, a.MemberName, 
    a.BirthDate, a.Address1, a.Address2, a.Phone, a.Email,
    a.EnrollmentDate, a.EnrollmentStore,
    a.TotalPointsEarned, a.TotalPointsBurned, a.TransactionTime, a.StoreLocation, 
    SUM(a.EarnedPoints) EarnedPoints, COUNT(DISTINCT a.RuleServiceTransactionID) TotalTransactions
FROM df AS a
GROUP BY a.MemberID, a.StoreLocation, DATE(a.TransactionTime)
HAVING COUNT(DISTINCT a.RuleServiceTransactionID) >= 3
;
"""
dfChannelA = sqldf(q, locals())

dfChannelA.iloc[:, [3,8,12]] = dfChannelA.iloc[:, [3,8,12]].apply(pd.to_datetime)



### Point B

In [16]:

### Point B - Poseidon - Earn More 150k
q = """ 
SELECT a.MemberID, a.LoyaltyMemberID, a.MemberName, a.BirthDate, a.Address1, a.Address2, a.Phone, a.Email,
    a.EnrollmentDate, a.EnrollmentStore, a.TotalPointsEarned, a.TotalPointsBurned, a.TransactionTime, a.StoreLocation, 
    SUM(a.EarnedPoints) EarnedPoints, COUNT(DISTINCT a.RuleServiceTransactionID) TotalTransactions
FROM (
    SELECT a.MemberID, a.LoyaltyMemberID, a.MemberName, 
        a.BirthDate, a.Address1, a.Address2, a.Phone, a.Email,
        a.EnrollmentDate, a.EnrollmentStore,
        a.TotalPointsEarned, a.TotalPointsBurned, a.TransactionTime, a.StoreLocation, 
        a.EarnedPoints, a.RuleServiceTransactionID
    FROM df AS a
    WHERE a.EarnedPoints > 150000 AND a.NewRetailValueBeforeTax > 0 
)as a
GROUP BY a.MemberID
;
"""
dfChannelB = sqldf(q, locals())

dfChannelB.iloc[:, [3,8,12]] = dfChannelB.iloc[:, [3,8,12]].apply(pd.to_datetime)



### Point C

In [18]:

### Point C - Asce - 1TimeIn3DaySameStores
# First Filter - Get >= 3 Trx
gclub_PointCandD = dfC.groupby(['MemberID']).agg({
    'RuleServiceTransactionID' : pd.Series.nunique
}).reset_index()

memberMore3 = gclub_PointCandD[gclub_PointCandD['RuleServiceTransactionID'] >=3]['MemberID']

rawC = dfC[dfC['MemberID'].isin(memberMore3)][['MemberID', 'StoreLocation', 'TransactionTime']].copy()
rawC['TransactionTime'] = rawC['TransactionTime'].dt.date
rawC['TransactionTime'] = pd.to_datetime(rawC['TransactionTime'])
rawC.drop_duplicates(subset=['MemberID', 'StoreLocation', 'TransactionTime'], keep='last', inplace=True)
rawC.sort_values(by=['MemberID', 'StoreLocation', 'TransactionTime'], inplace=True, ascending=False)
rawC.reset_index(drop=True, inplace=True)
# Search Per Day
memberPointC = []
# x = 0
for i in rawC['MemberID'].unique():
    df_1 = rawC[rawC['MemberID'] == i]
    for k in df_1['StoreLocation'].unique():
        x = 0
        df_2 = df_1[df_1['StoreLocation'] == k]
        for j in range(len(df_2)-1):
            if((df_2.iloc[(j), -1].day - df_2.iloc[(j+1), -1].day) == 1) :
                x+=1 
            else :
                x = 0
            if x == 2:
                memberPointC.append(i)
                break

df_memberPointC = pd.DataFrame(memberPointC, columns=['MemberID'])

q = """ 
SELECT a.MemberID, a.LoyaltyMemberID, a.MemberName, 
    a.BirthDate, a.Address1, a.Address2, a.Phone, a.Email,
    a.EnrollmentDate, a.EnrollmentStore,
    a.TotalPointsEarned, a.TotalPointsBurned, a.TransactionTime, a.StoreLocation, 
    SUM(a.EarnedPoints) EarnedPoints, COUNT(DISTINCT a.RuleServiceTransactionID) TotalTransactions
FROM dfC AS a
GROUP BY a.MemberID, a.StoreLocation
ORDER BY a.MemberID, a.StoreLocation
;
"""
dfChannelC = sqldf(q, locals())

dfChannelC.iloc[:, [3,8,12]] = dfChannelC.iloc[:, [3,8,12]].apply(pd.to_datetime)
dfChannelC = dfChannelC[dfChannelC['MemberID'].isin(df_memberPointC['MemberID'])].copy()



### Point D

In [20]:

### Point D - Enma - 4TimeDifferentStoreSameDay
q = """ 
SELECT a.MemberID, a.LoyaltyMemberID, a.MemberName, 
    a.BirthDate, a.Address1, a.Address2, a.Phone, a.Email,
    a.EnrollmentDate, a.EnrollmentStore,
    a.TotalPointsEarned, a.TotalPointsBurned, a.TransactionTime, a.StoreLocation, 
    SUM(a.EarnedPoints) EarnedPoints, COUNT(DISTINCT a.RuleServiceTransactionID) TotalTransactions
FROM df AS a
GROUP BY a.MemberID, DATE(a.TransactionTime)
HAVING COUNT(DISTINCT a.StoreLocation) >= 4
;
"""
dfChannelD = sqldf(q, locals())

dfChannelD.iloc[:, [3,8,12]] = dfChannelD.iloc[:, [3,8,12]].apply(pd.to_datetime)



### Point F & G

In [22]:

### Point F & G - Nika & Loki - 1AreaDifferentConcept & 1ConceptDiffrentStore
#### Raw Data F & G
dfPointFandG = gclub_SalesTrxValueEarnBurn[(gclub_SalesTrxValueEarnBurn['TransactionTime'] >= date_obj.strftime('%Y-%m-%d')) & 
                        (gclub_SalesTrxValueEarnBurn['TransactionTime'] <= date_obj.strftime('%Y-%m-%d'))][['MemberID', 
                            'RuleServiceTransactionID', 'TransactionTime', 'LocationReference', 'Area', 'ConceptName']].copy()   
                            
dfPointFandG = gclub_SalesTrxValueEarnBurn[(pd.to_datetime(gclub_SalesTrxValueEarnBurn['TransactionTime'].dt.date) >= str((date_obj + datetime.timedelta(days=-1)).strftime('%Y-%m-%d'))) & (pd.to_datetime(gclub_SalesTrxValueEarnBurn['TransactionTime'].dt.date) <= str((date_obj + datetime.timedelta(days=-1)).strftime('%Y-%m-%d')))][['MemberID', 'RuleServiceTransactionID', 'TransactionTime', 'LocationReference', 'Area', 'ConceptName']].copy()
                            
dfPointFandG.sort_values(by=['TransactionTime'], inplace=True)
dfPointFandG['TransactionTime'] = dfPointFandG['TransactionTime'].dt.date
dfPointFandG['TransactionTime'] = pd.to_datetime(dfPointFandG['TransactionTime'])

# Cleaning data to 1 concept name
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].apply(lambda x : 'Samsonite' if ((x == 'Samsonite DOS RPA') or (x == 'Samsonite MAP') or (x == 'Samsonite RPA')) else x)
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' RPA', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' MGI', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' MGR', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' BKMR', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' CMIR', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' MDFR', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' OFAR', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' PAR', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' PBPR', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' SDVR', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' SGR', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' SFAR', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' MFAR', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' MAP', '')
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].str.replace(' PFA', '')

dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].apply(lambda x : 'Kinokuniya' if (x == 'Kinokuniya Pustaka') else x)
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].apply(lambda x : 'Maxmara' if (x == 'Maxmara Weekend') else x)
dfPointFandG['ConceptName'] = dfPointFandG['ConceptName'].apply(lambda x : 'Tommy Hilfiger' if (x == 'Tommy Hilfiger ') else x)

# Drop data yang duplicate (seusai dengan channel yang diminta)
dfX = dfPointFandG.drop_duplicates(subset=['MemberID', 'Area', 'ConceptName', 'TransactionTime'])
dfY = dfPointFandG.drop_duplicates(subset=['MemberID', 'ConceptName', 'LocationReference', 'TransactionTime'])

# Gabungin unique concept name & ambil berapa total unique concept name nya
dfXGP1 = dfX.groupby(['MemberID', 'Area', 'TransactionTime'])['ConceptName'].apply(lambda x: ' | '.join(x)).reset_index()
dfXGP2 = dfX.groupby(['MemberID', 'Area', 'TransactionTime']).agg({'ConceptName' : pd.Series.nunique}).reset_index()

dfYGP1 = dfY.groupby(['MemberID', 'ConceptName', 'TransactionTime'])['LocationReference'].apply(lambda x: ' | '.join(x)).reset_index()
dfYGP2 = dfY.groupby(['MemberID', 'ConceptName', 'TransactionTime']).agg({'LocationReference' : pd.Series.nunique}).reset_index()

# Gabungin / left join 
dfXGP = pd.merge(dfXGP1, dfXGP2, on=['MemberID', 'Area', 'TransactionTime'])
dfYGP = pd.merge(dfYGP1, dfYGP2, on=['MemberID', 'ConceptName', 'TransactionTime'])

# rename column name
dfXGP.rename(columns={'ConceptName_x': 'ConceptName', 'ConceptName_y': 'NumConceptName'}, inplace=True)
dfYGP.rename(columns={'LocationReference_x': 'LocationReference', 'LocationReference_y': 'NumLocationReference'}, inplace=True)

# get only valid data (Point F)
dfXGP = dfXGP[dfXGP['NumConceptName'] >=3]
dfXGPF1 = dfXGP.groupby(['MemberID'])['Area'].apply(lambda x: ' <> '.join(x)).reset_index()
dfXGPF2 = dfXGP.groupby(['MemberID'])['ConceptName'].apply(lambda x: ' <> '.join(x)).reset_index()
dfXGPF3 = dfXGP.groupby(['MemberID']).agg({'Area' : pd.Series.nunique, 'NumConceptName' : np.sum}).reset_index() # catatan untuk numConcept agak tdk valid, karena tdk unique concept
dfXGPF = pd.merge(dfXGPF1, dfXGPF2, on=['MemberID'])
dfXGPF = pd.merge(dfXGPF, dfXGPF3, on=['MemberID'])
dfXGPF.rename(columns={'Area_x': 'Area', 'Area_y': 'NumArea'}, inplace=True)
dfXGPF = dfXGPF[dfXGPF['NumConceptName'] >= 3]
dfYGP = dfYGP[dfYGP['NumLocationReference'] >= 3]



### Get Point F & G

In [24]:

#### Point F
q = """ 
SELECT a.MemberID, a.LoyaltyMemberID, a.MemberName, 
    a.BirthDate, a.Address1, a.Address2, a.Phone, a.Email,
    a.EnrollmentDate, a.EnrollmentStore,
    a.TotalPointsEarned, a.TotalPointsBurned, a.TransactionTime, a.StoreLocation, 
    SUM(a.EarnedPoints) EarnedPoints, COUNT(DISTINCT a.RuleServiceTransactionID) TotalTransactions
FROM df AS a
GROUP BY a.MemberID
;
"""
dfChannelF = sqldf(q, locals())

dfChannelF.iloc[:, [3,8,12]] = dfChannelF.iloc[:, [3,8,12]].apply(pd.to_datetime)
dfChannelF = dfChannelF[dfChannelF['MemberID'].isin(dfXGPF['MemberID'])].copy()

#### Point G
q = """ 
SELECT a.MemberID, a.LoyaltyMemberID, a.MemberName, 
    a.BirthDate, a.Address1, a.Address2, a.Phone, a.Email,
    a.EnrollmentDate, a.EnrollmentStore,
    a.TotalPointsEarned, a.TotalPointsBurned, a.TransactionTime, a.StoreLocation, 
    SUM(a.EarnedPoints) EarnedPoints, COUNT(DISTINCT a.RuleServiceTransactionID) TotalTransactions
FROM df AS a
GROUP BY a.MemberID
;
"""
dfChannelG = sqldf(q, locals())

dfChannelG.iloc[:, [3,8,12]] = dfChannelG.iloc[:, [3,8,12]].apply(pd.to_datetime)
dfChannelG = dfChannelG[dfChannelG['MemberID'].isin(dfYGP['MemberID'])].copy()



## Status Member

In [26]:

### Status All Member
q = """ 
SELECT a.LoyaltyMemberID, a.Status
FROM (
SELECT DISTINCT a.LoyaltyMemberID, '3TSS' AS Status
FROM dfChannelA AS a
UNION ALL 
SELECT DISTINCT a.LoyaltyMemberID, 'E150K'
FROM dfChannelB AS a
UNION ALL 
SELECT DISTINCT a.LoyaltyMemberID, '1T3DSS' AS Status
FROM dfChannelC AS a
UNION ALL 
SELECT DISTINCT a.LoyaltyMemberID, '4TDS' AS Status
FROM dfChannelD AS a
UNION ALL 
SELECT DISTINCT a.LoyaltyMemberID, '1ADC'
FROM dfChannelF AS a
UNION ALL 
SELECT DISTINCT a.LoyaltyMemberID, 'SCDS' AS Status
FROM dfChannelG AS a
) AS a 
;
"""
statusMember = sqldf(q, locals())
statusMemberGP1 = statusMember.groupby(['LoyaltyMemberID'])['Status'].apply(lambda x: ','.join(x)).reset_index()
statusMemberGP2 = statusMember.groupby(['LoyaltyMemberID']).agg({'Status' : pd.Series.nunique}).reset_index()
dfStatusGP = pd.merge(statusMemberGP1, statusMemberGP2, on=['LoyaltyMemberID'])
dfStatusGP.rename(columns={'Status_x': 'Status', 'Status_y': 'NumStatus'}, inplace=True)
## Final Recap (To Excel & Status) - Excel Part 1
dfChannelF = dfChannelF.merge(dfXGPF,left_on='MemberID',right_on='MemberID', how='left')
dfChannelG = dfChannelG.merge(dfYGP,left_on='MemberID',right_on='MemberID', how='left')
dfChannelG.drop_duplicates(subset='MemberID', inplace=True)

# Every Channel Give Status
dfChannelA['Channels'] = '3TSS'
dfChannelB['Channels'] = 'E150K'
dfChannelC['Channels'] = '1T3DSS'
dfChannelD['Channels'] = '4TDS'
dfChannelF['Channels'] = '1ADC'
dfChannelG['Channels'] = 'SCDS'



## Cleaning Data

### Rename Columns

In [29]:

# Rename columns data
dataGabunganA = pd.concat([dfChannelA,dfChannelB,dfChannelC,dfChannelD])
dataGabunganA.drop(columns=['Address2'], axis=1, inplace=True)
dataGabunganA.rename(columns={
    'MemberName' : 'Name',
    'Address1' : 'Address',
    'TotalPointsEarned' :  'Points Earned',
    'TotalPointsBurned' : 'Points Burned',
    'TransactionTime' : 'TransactionDate',
    'EarnedPoints' : 'Base Earned Points'
}, inplace=True)

# Rename columns data
dfChannelF.drop(columns=['Address2', 'NumArea'], axis=1, inplace=True)
dfChannelF.rename(columns={
    'MemberName' : 'Name',
    'Address1' : 'Address',
    'TotalPointsEarned' :  'Points Earned',
    'TotalPointsBurned' : 'Points Burned',
    'TransactionTime' : 'TransactionDate',
    'EarnedPoints' : 'Base Earned Points',
    'Area' : 'Types',
    'ConceptName' : 'Concept',
    'NumConceptName' : 'NumConcept'
}, inplace=True)

dfChannelG.drop(columns=['Address2'], axis=1, inplace=True)
dfChannelG.rename(columns={
    'MemberName' : 'Name',
    'Address1' : 'Address',
    'TotalPointsEarned' :  'Points Earned',
    'TotalPointsBurned' : 'Points Burned',
    'TransactionTime' : 'TransactionDate',
    'StoreLocation_x' : 'StoreLocation',
    'EarnedPoints' : 'Base Earned Points',
    'ConceptName' : 'Types',
    'StoreLocation_y' : 'Concept',
    'NumStoreLocation' : 'NumConcept'
}, inplace=True)

dataGabunganB = pd.concat([dfChannelF,dfChannelG])

# Concat Data All Channel
dataGabungan = pd.concat([dataGabunganA,dataGabunganB])



### Get Snap Data & Etc

In [31]:

# For get data date date snap
dataGabungan['Week'] = date_obj.isocalendar()[1]
dataGabungan['Month'] = date_obj.month
dataGabungan['Year'] = date_obj.year
dataGabungan['DateSnap'] = date_obj

# Reposition Column Form
dataGabungan = dataGabungan[['Week', 'Month', 'Year', 'DateSnap','MemberID', 'LoyaltyMemberID', 'Name', 'BirthDate', 'Address', 'Phone',
    'Email', 'EnrollmentDate', 'EnrollmentStore', 'Points Earned', 'Points Burned', 'TransactionDate', 'StoreLocation',
    'Base Earned Points', 'TotalTransactions', 'Channels', 'Types', 'Concept', 'NumConcept']]

# Drop duplicate by weekly
dataGabungan.drop_duplicates(subset=['Week', 'MemberID'], inplace=True)

# Left join data member to all status channel member
dataGabungan = dataGabungan.merge(dfStatusGP,on='LoyaltyMemberID',how='left')

# Change data type
dataGabungan['LoyaltyMemberID'] = dataGabungan['LoyaltyMemberID'].astype('str')



### Enroll Age Year

In [33]:

## Update Auto Fraud (Part 2) - Excel Part 3
# Enroll Age Year
def crtiX(x):
    if x < 1 : return '1. <1 Year'
    elif x < 2 : return '2. 1-2 Year'
    elif x < 3 : return '3. 2-3 Year'
    else : return '4. >3 Year'

dataGabungan['EnrollYearsCrit'] = dataGabungan['EnrollmentDate'].apply(lambda x : date_obj - x.date()) / np.timedelta64(1, 'Y')
dataGabungan['EnrollYearsCrit'] = dataGabungan['EnrollYearsCrit'].apply(lambda x : crtiX(x))


### True / False - Status Criteria

In [35]:

dataGabungan['3TSS'] = dataGabungan['Status'].str.contains('3TSS')
dataGabungan['E150K'] = dataGabungan['Status'].str.contains('E150K')
dataGabungan['1T3DSS'] = dataGabungan['Status'].str.contains('1T3DSS')
dataGabungan['4TDS'] = dataGabungan['Status'].str.contains('4TDS')
dataGabungan['1ADC'] = dataGabungan['Status'].str.contains('1ADC')
dataGabungan['SCDS'] = dataGabungan['Status'].str.contains('SCDS')


## Inject Data to Database

In [37]:

# Insert data weekly to data master (sql)
dataGabungan.to_sql(con=connect_string_dwhdb,
    name='dfFraudMasterFinal',
    schema='gclub', # re-check
    index=False,
    if_exists='append')

## Send Email

In [39]:

import datetime
import smtplib
import base64
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

def sendEmailSP(data):
    # Email configuration
    sender_email = "dp-alert@notify.gtech.digital"
    sender_password = ""
    receiver_email = "ferdinand.tongga@gtech.digital, fazri.fatahillah@mapclub.com"
    #edited by rey 7/17/2023
    # receiver_email = "attariq.kasfilla@mapclub.com, reyki.seprianza@gtech.digital, fazri.fatahillah@mapclub.com"
    # cc_receiver_email = "michael.danendra@mapclub.com, budi.santoso@mapclub.com, vincent.iskandar@gtech.digital, rian.andrea@gtech.digital, ferdinand.tongga@gtech.digital"
    subject = "Fraud Detection - {0}".format((date_obj).strftime("%Y-%m-%d"))
    body = "Dear All, \nBerikut results fraud detection per hari ini yang perlu segera di analisa\n4 Detection = {0} Member\n5 Detection = {1} Member\n6 Detection = {2} Member".format(data[data['NumStatus'] == 4]['MemberID'].nunique(),
                                     data[data['NumStatus'] == 5]['MemberID'].nunique(),
                                     data[data['NumStatus'] == 6]['MemberID'].nunique())

    # Excel attachment configuration
    attachment_path = 'DataFraudDetection.xlsx'

    # Compose the email
    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = subject
    # msg["Cc"] = cc_receiver_email

    msg.attach(MIMEText(body, "plain"))

    # Attach the Excel file
    with open(attachment_path, "rb") as f:
        attachment = MIMEApplication(f.read(), _subtype="xlsx")
        attachment.add_header("Content-Disposition", "attachment", filename=attachment_path)
        msg.attach(attachment)

    # Send the email
    with smtplib.SMTP('smtpdm-ap-southeast-1.aliyun.com', 80) as server:
        # server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(msg)
        print("Email sent successfully!")


if True in (dataGabungan['NumStatus'].unique() >= 4):
    saveToExcelForEmail = dataGabungan[dataGabungan['NumStatus'] >= 4].copy()
    saveToExcelForEmail.to_excel('DataFraudDetection.xlsx', index=False)
    sendEmailSP(saveToExcelForEmail)
else : print('ngga ada')

In [40]:
%sh
ls

In [41]:
%sh
rm DataFraudDetection.xlsx